### Find all DEGs

In [1]:
library('glmGamPoi')
library('Seurat')
library('stringr')
library('dplyr')
library('tidyr')
library('ggplot2')
library('pheatmap')
library('magrittr')
library('SingleCellExperiment')
library('ggrepel')
options(repr.plot.width = 7, repr.plot.height = 7) # set default plot size
options(jupyter.plot_mimetypes = 'image/svg+xml') # output SVG
options(ggrepel.max.overlaps = Inf)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t



Attaching package: ‘dplyr’


The following object is masked from ‘package:glmGamPoi’:

    vars


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘ggplot2’


The following object is masked from ‘package:glmGamPoi’:

    vars



Attaching package: ‘magrittr’


The following object is masked from ‘package:tidyr’:

    extract


Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, co

In [ ]:
CRISPRi_seurat5 <- readRDS('~/data/seurat/CRISPRi_seurat5.rds')
CRISPRa_seurat5 <- readRDS('~/data/seurat/CRISPRa_seurat5.rds')

In [ ]:
# keep only protein coding genes
annot_human = read.table("~/data/Biomart_Human_Genes_Ensembl101.txt",sep="\t", header=T)
protein_coding_genes = annot_human[annot_human$gene_biotype == 'protein_coding','external_gene_name']

In [ ]:
CRISPRi_seurat5 <- subset(CRISPRi_seurat5, features = protein_coding_genes)
CRISPRa_seurat5 <- subset(CRISPRa_seurat5, features = protein_coding_genes)

In [ ]:
sce_CRISPRi = as.SingleCellExperiment(CRISPRi_seurat5)
sce_CRISPRa = as.SingleCellExperiment(CRISPRa_seurat5)

In [ ]:
genesA = read.csv('~/data/CRISPRa_gene_list.csv')
genesA = genesA$x

genesI = read.csv('~/data/CRISPRi_gene_list.csv')
genesI = genesI$x

In [ ]:
CRISPRi_DEGenes = list()

for(gene in genesI){
    #lower counts cutoff
    sce_subset <- sce_CRISPRi[rowSums(counts(sce_CRISPRi)) > 100, sce_CRISPRi$Gene_Targeted %in% c(gene, "NT")]
    assays(sce_subset)[['logcounts']] <- NULL

    fit <- glm_gp(sce_subset, design = ~ Gene_Targeted + donor - 1, reference_level="NT", on_disk=FALSE)
    genename = paste0('`Gene_Targeted',gene,"`")
    res <- test_de(fit, eval(parse(text = genename)) - `Gene_TargetedNT`, sort_by = pval)
    CRISPRi_DEGenes[[gene]] = res
    
}

In [ ]:
CRISPRa_DEGenes = list()

for(gene in genesA){
    #lower counts cutoff
    sce_subset <- sce_CRISPRa[rowSums(counts(sce_CRISPRa)) > 100, sce_CRISPRa$Gene_Targeted %in% c(gene, "NT")]
    assays(sce_subset)[['logcounts']] <- NULL

    fit <- glm_gp(sce_subset, design = ~ Gene_Targeted + donor - 1, reference_level="NT", on_disk=FALSE)
    genename = paste0('`Gene_Targeted',gene,"`")
    res <- test_de(fit, eval(parse(text = genename)) - `Gene_TargetedNT`, sort_by = pval)
    CRISPRa_DEGenes[[gene]] = res
    
}

In [ ]:
for (gene in genesI){
    degs1 = CRISPRa_DEGenes[[gene]]
    degs2 = CRISPRi_DEGenes[[gene]]
    write.csv(degs1, paste0('~/data/DEGs/CRISPRa/', gene, '.csv'))
    write.csv(degs2, paste0('~/data/DEGs/CRISPRi/', gene, '.csv'))
}